In [1]:
import pint
from tinarm import NameQuantityPair
from tinarm import Quantity as OldQuantity
from tinarm import Unit as OldUnit
import logging
import time
import requests
import uuid
import random

LOGGING_LEVEL = logging.INFO


### Configure Logging
logger = logging.getLogger()
logger.setLevel(LOGGING_LEVEL)

q = pint.UnitRegistry()


Log in to Tin Arm Engineering's machine solver, and under profile, retrieve your API key. 
Do not commit your API key to a repository, and consider it like a password.  A good way to keep it out of this code base is to use a configuation file, or environment variable. We will want other things cofigured too, so let's use a yaml file for convenience. 

create a `configurations.yaml` file in this directory with the content.
```yaml
api_key: 668952058c490d0a296da0abb966****
root_url: http://server-go:4300
```
But of course replace the number with your api key

You can then add that file to your `.gitignore` file by executing ```echo 'configurations.yaml' >> .gitignore```

```
!echo 'configurations.yaml' >> .gitignore
```

In [2]:
import yaml

with open("configurations.yaml", "r") as f:
    config = yaml.safe_load(f)



# Stator

In [3]:
stator_parameters = {
    "slot_liner_thikness": 0.2 * q.mm,
    "stator_bore": 82.0 * q.mm,
    "tooth_tip_depth": 1.5 * q.mm,
    "slot_opening": 4 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 136 * q.mm,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 1 * q.mm,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

## Rotor
Surface mounted Breadloaf magnets

In [4]:
rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

## Simulation Parameters

In [5]:
simulation_parameters = {
       "samples_per_electrical_period": 180 * q.count,
        "timestep_intervals": 180 * q.count,
        "active_length": 65 * q.mm}


In [6]:
winding_parameters = {
    "symmetry": 2 * q.count,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

## Operating Point

In [7]:
op_parameters = {
    "simulated_speed": 2060 * q.rpm,
    "current_density": 0 * q.A / q.mm ** 2,
    "current_angle": 0 * q.degrees
    }


In [8]:
class Unit:
    def __init__(self, name: str, exponent: int):
        self.name = name
        self.exponent = int(exponent)

    def to_dict(self):
        return {
            "name": self.name,
            "exponent": self.exponent,
        }
    
class Quantity:
    def __init__(self, magnitude, units: list[Unit]):
        self.magnitude = magnitude
        self.units = [Unit(*u) for u in units]

    def to_dict(self):
        return {
            "magnitude": self.magnitude,
            "units": [u.to_dict() for u in self.units],
        }
    


In [9]:
[stator_parameters[k].to_base_units().to_tuple() for k in  stator_parameters]

[(0.0002, (('meter', 1.0),)),
 (0.082, (('meter', 1.0),)),
 (0.0015, (('meter', 1.0),)),
 (0.004, (('meter', 1.0),)),
 (0.009800000000000001, (('meter', 1.0),)),
 (0.136, (('meter', 1.0),)),
 (0.0055, (('meter', 1.0),)),
 (0.001, (('meter', 1.0),)),
 (12, (('count', 1),)),
 (1.2217304763960306, (('radian', 1),))]

In [10]:
[stator_parameters[k].to_tuple() for k in  stator_parameters]

[(0.2, (('millimeter', 1),)),
 (82.0, (('millimeter', 1),)),
 (1.5, (('millimeter', 1),)),
 (4, (('millimeter', 1),)),
 (9.8, (('millimeter', 1),)),
 (136, (('millimeter', 1),)),
 (5.5, (('millimeter', 1),)),
 (1, (('millimeter', 1),)),
 (12, (('count', 1),)),
 (70, (('degree', 1),))]

In [11]:
class Machine(object):
    def __init__(self, stator_parameters, rotor_parameters, winding_parameters):
        self._stator_api = [NameQuantityPair("stator",
                                        k,
                                        Quantity(*stator_parameters[k].to_tuple())) for k in  stator_parameters]
        self.stator = stator_parameters

        self._rotor_api = [NameQuantityPair("rotor",
                                        k,
                                        Quantity(*rotor_parameters[k].to_tuple())) for k in  rotor_parameters]
        self.rotor = rotor_parameters
        self._winding_api = [NameQuantityPair("winding",
                                        k,
                                        Quantity(*winding_parameters[k].to_tuple())) for k in  winding_parameters]
        self.winding = winding_parameters
    def __repr__(self) -> str:
        return f"Machine({self.stator_parameters}, {self.rotor_parameters}, {self.winding_parameters})"
    def to_dict(self):
        data = []
        data.extend(list(x.to_dict() for x in self._stator_api))
        data.extend(list(x.to_dict() for x in self._rotor_api))
        data.extend(list(x.to_dict() for x in self._winding_api))
        return data
        
    
class Job(object):
    def __init__(self, machine: Machine, op_parameters, simulation_parameters, title=None):
        if title is None:
            self.title = self.generate_title()
        else:
            self.title = title
        self.type = "electromagnetic_spmbrl_fscwseg"
        self.status = 0
        self.machine = machine

        self.operating_point = op_parameters
        self._operating_point_api = [NameQuantityPair("operating_point",
                                        k,
                                        Quantity(*op_parameters[k].to_tuple())) for k in  op_parameters]
        
        self.simulation = simulation_parameters
        self._simulation_api = [NameQuantityPair("simulation",
                                        k,
                                        Quantity(*simulation_parameters[k].to_tuple())) for k in  simulation_parameters]
    def __repr__(self) -> str:
        return f"Job({self.machine}, {self.operating_point}, {self.simulation})"
    
    def generate_title(self):
        "gets a random title from the wordlists"
        random_offset = random.randint(500, 286797)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Adjectives-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        adjective = random.choice(response.text.splitlines()[1:-1])
        random_offset = random.randint(500, 871742)
        response = requests.get("https://github.com/taikuukaits/SimpleWordlists/raw/master/Wordlist-Nouns-All.txt",
                                headers={'Range': 'bytes={1}-{0}'.format(random_offset, random_offset-500), 'accept-encoding': 'identity'})
        noun = random.choice(response.text.splitlines()[1:-1])
        title = f"{adjective}-{noun}"
        return title 

    def to_dict(self):
        job =  {"status": 0,
                "title": self.title,
                "type": self.type,
                "tasks": 11,
                "data": []}
        
        job["data"].extend(list(x.to_dict() for x in self._operating_point_api))
        job["data"].extend(list(x.to_dict() for x in self._simulation_api))
        job["data"].extend(self.machine.to_dict())
        return job
    
    def run(self):
        pass

In [12]:
m1 = Machine(stator_parameters, rotor_parameters, winding_parameters)

j1 = Job(m1, op_parameters, simulation_parameters)


In [13]:
class Api:
    """
    The TAE User API
    """

    def __init__(self, root_url, api_key):
        """
        Initialize the API
        """
        self._root_url = root_url
        self._api_key = api_key
        self._org_id = "659bf5a7b7135efed48311ba"

        logger.info(f"root_url: {self._root_url}")

    def get_job(self, job_id):
        """
        Get a job from the TAE API
        """
        response = requests.get(
            url=f"{self._root_url}/jobs/{job_id}?apikey={self._api_key}",
        )
        response.raise_for_status()
        return response.json()
    
    def create_job(self, job):
        """
        Create a job for the TAE API
        """
        response = requests.post(
            url=f"{self._root_url}/jobs/?apikey={self._api_key}&org_id={self._org_id}",
            json=job.to_dict()
        )
        response.raise_for_status()
        return response.json()
    

In [14]:
api = Api(config["root_url"], config["api_key"])

2024-01-17 19:59:46,705 - NoJobId - INFO - Martins-MacBook-Air.local - 3557394831.py->__init__() - root_url: https://api.build.tinarmengineering.com


In [15]:
api.create_job(j1)

{'id': '65a823a33ef353f6703a580b',
 'owner_org_id': '659bf5a7b7135efed48311ba',
 'owner_user_id': '659bf5a7b7135efed48311bd',
 'creation_date': '2024-01-17T18:59:47Z',
 'status': 0,
 'title': 'uncontested-yaws',
 'type': 'electromagnetic_spmbrl_fscwseg',
 'data': [{'section': 'operating_point',
   'name': 'simulated_speed',
   'value': {'magnitude': 215.72269554649912,
    'units': [{'name': 'radian', 'exponent': 1},
     {'name': 'second', 'exponent': -1}]}},
  {'section': 'operating_point',
   'name': 'current_density',
   'value': {'magnitude': 0,
    'units': [{'name': 'ampere', 'exponent': 1},
     {'name': 'meter', 'exponent': -2}]}},
  {'section': 'operating_point',
   'name': 'current_angle',
   'value': {'magnitude': 0, 'units': [{'name': 'radian', 'exponent': 1}]}},
  {'section': 'simulation',
   'name': 'samples_per_electrical_period',
   'value': {'magnitude': 180, 'units': [{'name': 'count', 'exponent': 1}]}},
  {'section': 'simulation',
   'name': 'timestep_intervals',
  